# Práctica 1

## Preparación del entorno

In [1]:
import numpy as np
import tensorflow as tf
import keras as ker
import matplotlib.pyplot as plt 
import pandas as pd
import sklearn as sk
import PIL
import os
import random

from PIL import Image
from IPython.display import clear_output # Limpiar output
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from numpy import asarray
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm

### Variables a definir

In [3]:
# No tocar
num_clases = 10
# La foto es 128x96
xpixel = 128
ypixel = 96
# Tocar
n_neuronas_conv1 = 32
n_neuronas_conv2 = 64
n_neuronas_conv3 = 128
l_rate = 0.1
epoch = 12 
batch = 16

# Preprocesamiento de datos

### Nuestro primer aproach no funciona

In [38]:
def crear_data_set_entreno(): # Besitos gonzalo de parte de Ini y Marcos <3
    # TECNICAMENTE, CUANDO USAMOS AUGMENTATION SE TIENEN QUE USAR SOLO LOS DATOS AUMENTADOS, AUN ASI, NOSOTROS MEZCLAMOS AMBOS, POR SI ACASO ESO MEJORA LA RED
# Creamos un set entero de entrenamiento. A posterior este set lo dividiremos en entrenamiento y validación

# Esto es para decir como queremos las nuevas imagenes
    augmentation = ImageDataGenerator( # Esto lo vamos a tener que poner todo seguido sin comentar para que no salga plagio !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        rescale=1/255.,              # normalizar                     > Es posible que esto no sea necesario ya que ya esta normalizado
        brightness_range=[0.1, 0.7], # luminosidad
        width_shift_range=0.5,       # cambiar anhcura 50%
        rotation_range=0,            # rotar
        shear_range = 0.2,           # idk
        zoom_range=0.2,              # zoom
        horizontal_flip=False,        # 180º horizontal
        vertical_flip=False#,          # 180º vertical
        #validation_split=0.2        # usamos el 20% de los datos por epoch
        )
    # Esto creaa dichas imagenes apartir del directorio que queramos

    dataset_entreno_augmentation = augmentation.flow_from_directory(
            directory=r"../Dataset/imgs/train/",
            target_size=(128, 96), #Queremos tener imagenes de la misam resolucion
            color_mode="rgb",
            batch_size=1, # N_img / batch_size = N_nuevas_imagenes
            class_mode="categorical",
            shuffle=True,
            seed=42
    )


    # Ahora creamos el set de entrenamiento con las imagenes originales
    dataset_entreno_original = tf.keras.utils.image_dataset_from_directory(
    directory = r"../Dataset/imgs/train/",
    seed=123,
    image_size=(128, 96),
    batch_size=32
    )
    ### Nos falta añadir las de entreno aparte de las obtenidas por augmentation
    # Juntamos los sets de augmentation (las imagenes nuevas) con el set de las originales (las que nos dan)
    datasetFinal = dataset_entreno_original, dataset_entreno_augmentation
    # Separamos todo en entreno y validacion
    entreno, validacion = train_test_split(datasetFinal,test_size = 0.20, shuffle=True) # El entreno es el 80% y el conjunto prueba 20% para el set de augmentation
    #entreno_orig_only, validacion_orig_only = train_test_split(dataset_entreno_original,test_size = 0.20, shuffle=True)
    entreno_aug_only,validacion_aug_only = train_test_split(dataset_entreno_augmentation,test_size = 0.20, shuffle=True)
    #return entreno,validacion,entreno_orig_only,validacion_orig_only,entreno_aug_only,validacion_aug_only 
    return entreno,validacion,entreno_aug_only,validacion_aug_only 

entreno,validacion,entreno_aug_only,validacion_aug_only  = crear_data_set_entreno()
#entreno,validacion,entreno_orig_only,validacion_orig_only,entreno_aug_only,validacion_aug_only  = crear_data_set_entreno()


'''mirar respuestas-> https://stackoverflow.com/questions/57092637/how-to-fit-keras-imagedatagenerator-for-large-data-sets-using-batches'''

Found 22424 images belonging to 10 classes.
Found 22424 files belonging to 10 classes.


### Esto coge los datos originales y los divide parece funcionar para model.fit

In [8]:
# Esto funciona con fit pero necesito investigar

# Mi  teoria de lo que es esto por lo que he leido (OJO NO LO SE REALMENTE) esto crea iteradores que formatean la imagen y la pasan por al model.fit el batch size es en cuanto pedazos pasas
# por cadda iteracion como diferencia y hace el split entre validadcion y training no lose pero salia asi en bastantes sitios 

batch_size = 16

# this is the augmentation configuration we will use for training     ESTO define como pasas las imagenes
train_datagen = ImageDataGenerator(
        rescale=1./255, 
        shear_range=0.2,
        zoom_range=0.2,
        validation_split=0.2,
        horizontal_flip=True)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator_t = train_datagen.flow_from_directory(                        # Esto creo que crea un iterador en el directorio que sige las intrucciones mencioanda arriba para pasar los
                                                                                # datos a model.fit =/ nose es una paranoia
        '../Dataset/imgs/train/',  # this is the target directory
        color_mode='rgb',
        target_size=(128, 96),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        subset="training")  # since we use binary_crossentropy loss, we need binary labels

train_generator_v = train_datagen.flow_from_directory(
        '../Dataset/imgs/train/',  # this is the target directory
        color_mode='rgb',
        target_size=(128, 96),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        subset="validation") 


Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


##### Otra Red para no borrar lo ya hecho

In [40]:
def Modelar_red(n_conv1,n_conv2,n_conv3,x,y,lr):
    model = Sequential()

    #### Input Layer ####
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',
                    activation='relu', input_shape=(128, 96, 3)))

    #### Convolutional Layers ####
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))  # Pooling
    model.add(Dropout(0.2)) # Dropout

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(ker.layers.Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (5,5), padding='same', activation='relu'))
    model.add(Conv2D(512, (5,5), activation='relu'))
    model.add(MaxPooling2D((4,4)))
    model.add(Dropout(0.2))

    #### Fully-Connected Layer ####
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax')) #len(class_subset) en vez de 10, lo he cambiado pero no estoy seguro

    # Compilamos
    adam = ker.optimizers.Adam(learning_rate= lr)
    model.compile(loss=ker.loss.categorical_crossentropy,optimizer=adam,metrics=['accuracy'])

    model.summary() # a handy way to inspect the architecture
    return model

### Red OG

In [9]:
def Modelar_red(n_conv1,n_conv2,n_conv3,x,y,lr):
    model=Sequential()
    # Capa input
    model.add(Conv2D(filters= n_conv1, kernel_size=(3,3),input_shape=(x,y,3),padding='same',activation='relu',kernel_constraint=maxnorm(3))) 
    # Capas convolucionales
    #   >este bloque se puede seguir añadiendo, quiza con menos neuronas, o menos capas convolucionales, pongo dos por dar un ejemplo nada mas
    # (3,3) es mucho se recomienda unsar 1x1 cuando las img no son mayores de 128x128 la nuestra es 128x96
    model.add(Conv2D(n_conv2,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) # pooling
    model.add(Dropout(0.2)) # dropout

    model.add(Conv2D(n_conv3,(3,3),activation='relu',padding='same')) 
    model.add(Conv2D(n_conv3,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) 
    model.add(Dropout(0.2)) 

    # Capa fully-connected - MLP
    model.add(Flatten())
    model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3))) # red fully-connected
    model.add(Dropout(0.5))
    model.add(Dense(num_clases, activation='softmax')) # capa de salida(softmaxx) 

    print(model.summary()) # con esto inspeccionamos el modelo, muy comodo

    # Compilamos
    adam = tf.keras.optimizers.Adam(learning_rate= lr) # https://keras.io/api/optimizers/adam/ ?
    model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=['accuracy'])

    return model
model = Modelar_red(n_neuronas_conv1,n_neuronas_conv2,n_neuronas_conv3,xpixel,ypixel,l_rate)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 96, 32)       896       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 96, 64)       18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 48, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 64, 48, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 48, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 48, 128)       147584    
                                                        

### Y ahora realizamos el fit para entrenar la red

In [10]:
def Entrenar(m,e,v,epo,b): #(model,entrenamiento,validacion, epoch,batch)
        m.fit(e,
        validation_data=v,
                use_multiprocessing=True,
                workers=6, # Esta parte es para que se separen 6 threads paralelos gracias a fit_generator
                epochs= epo,
                batch_size = 16,
                verbose=False
        ) 
        return m
model = Entrenar(model,train_generator_t,train_generator_v,epoch,batch)
# DATO CURIOSO, .FIT() TIENE UNA OPCION LLAMADA VALIDATION_DATA=, IGUAL NI ES NECESARIO CREAR UN GRUPO DE VALIDACION A PARTE

Exception in thread Thread-116:
Traceback (most recent call last):
  File "C:\Users\inigo\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\inigo\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\inigo\anaconda3\lib\site-packages\keras\utils\data_utils.py", line 745, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\inigo\anaconda3\lib\site-packages\keras\utils\data_utils.py", line 722, in pool_fn
    pool = get_pool_class(True)(
  File "C:\Users\inigo\anaconda3\lib\multiprocessing\context.py", line 119, in Pool
    return Pool(processes, initializer, initargs, maxtasksperchild,
  File "C:\Users\inigo\anaconda3\lib\multiprocessing\pool.py", line 212, in __init__
    self._repopulate_pool()
  File "C:\Users\inigo\anaconda3\lib\multiprocessing\pool.py", line 303, in _repopulate_pool
    return self._repopulate_pool_static(self._ctx, self.Process,
 

### Evaluamos el modelo con los datos de test (si hay...)

In [ ]:
def Evaluar(m,e_test,s_test): # model,entrada_test,salida_test
    return m.evaluate(e_test,s_test,verbose=False)
resultado = Evaluar(model, entrada_test, salida_test) # no existen estas variables todavia

### Predecir:

In [ ]:
prediccion = model.predict(x_test) # no existe esta variable todavia
print(prediccion)